In [8]:
import os
os.environ['PATH'] += ":/content/mumax3.10"

In [9]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import Rbf

In [10]:
def read_mumax3_table(filename):
    """Puts the mumax3 output table in a pandas dataframe"""

    from pandas import read_table
    
    table = read_table(filename)
    table.columns = ' '.join(table.columns).split()[1::2]
    
    return table

In [11]:
def read_mumax3_ovffiles(outputdir):
    """Load all ovffiles in outputdir into a dictionary of numpy arrays 
    with the ovffilename (without extension) as key"""
    
    from subprocess import run, PIPE, STDOUT
    from glob import glob
    from os import path
    from numpy import load

    # convert all ovf files in the output directory to numpy files
    p = run(["mumax3-convert","-numpy",outputdir+"/*.ovf"], stdout=PIPE, stderr=STDOUT)
    if p.returncode != 0:
        print(p.stdout.decode('UTF-8'))

    # read the numpy files (the converted ovf files)
    fields = {}
    for npyfile in glob(outputdir+"/*.npy"):
        key = path.splitext(path.basename(npyfile))[0]
        fields[key] = load(npyfile)
    
    return fields

In [12]:
def run_mumax3(script, name, verbose=False):
    """ Executes a mumax3 script and convert ovf files to numpy files
    
    Parameters
    ----------
      script:  string containing the mumax3 input script
      name:    name of the simulation (this will be the name of the script and output dir)
      verbose: print stdout of mumax3 when it is finished
    """
    
    from subprocess import run, PIPE, STDOUT
    from os import path

    scriptfile = name + ".txt" 
    outputdir  = name + ".out"

    # write the input script in scriptfile
    with open(scriptfile, 'w' ) as f:
        f.write(script)
    
    # call mumax3 to execute this script
    p = run(["mumax3","-f",scriptfile], stdout=PIPE, stderr=STDOUT)
    if verbose or p.returncode != 0:
        print(p.stdout.decode('UTF-8'))
        
    if path.exists(outputdir + "/table.txt"):
        table = read_mumax3_table(outputdir + "/table.txt")
    else:
        table = None
        
    fields = read_mumax3_ovffiles(outputdir)
    
    return table, fields

In [ ]:
# NUMERICAL PARAMETERS
fmax = 0.1e9       # maximum frequency (in Hz) of the sinc pulse
T    = 400e-9      # simulation time (longer -> better frequency resolution)
dt   = 250e-12     # the sample time

script=f"""
// Racetrack crystal settings:
length := 200e-9
width := 25e-9
delta := 100e-9
lattice_constant := 200e-9
N_unitcells := 8
Nx := 100
Ny := 256
Nz := 1
Lx := length
Ly := N_unitcells * lattice_constant
dX := length / Nx
dY := Ly / Ny
dZ := 3e-9 / Nz

setGridSize(Nx, Ny, Nz)
setCellSize(dX, dY, dZ)

// Material Constants
Bz := 0.01
Msat = 1273e3
Aex = 10e-12
anisU = vector(0, 0, 1)
Ku1 = 1.5e6
alpha = 0
Xi = 0.2
demagAccuracy = 8
pinning := false

two_racetrack := rect(length, delta+width)
racetrack := rect(length, width)

gap := delta - width

singleR := racetrack.repeat(0, lattice_constant, 0).transl(0, width/2, 0)
twoR := singleR.add(singleR.transl(0, delta, 0))

setgeom(twoR)

// Define regions
defRegion(1, singleR)
defRegion(2, singleR.transl(0, delta, 0))
defRegion(3, racetrack.transl(0, -Ly/2+width/2, 0))
save(regions)

B_ext.setregion(3, vector(0, 0, Bz*sinc(2*pi*{fmax}*(t-{T}/2))))

m.setRegion(1, twoDomain(0, 0, 1, 1, 1, 0, 0, 0, -1))
m.setRegion(2, twoDomain(0, 0, -1, 1, 1, 0, 0, 0, 1))
m.setRegion(3, twoDomain(0, 0, 1, 1, 1, 0, 0, 0, -1))
setPBC(0, 2, 0)
relax()

// Schedule output & save results
autosave(m, {dt})
tableadd(e_total)
tableautosave({dt})

// Run for 1ns with current through the sample
temp.set(1)
j = vector(0, 0, 0)
pol = 1
run({T})
"""

In [ ]:
table, fields = run_mumax3( script, name="DW_Lattice", verbose=False )

In [ ]:
'''
Extract table and fields, after simulation has run
'''
name = "dwlattice/micromagnetics/DW_Lattice"

scriptfile = name + ".txt" 
outputdir  = name + ".out"

table = read_mumax3_table(outputdir + "/table.txt") 
fields = read_mumax3_ovffiles(outputdir)

In [13]:
print(table)

plt.figure()

nanosecond = 1e-9
plt.plot( table["t"]/nanosecond, table["mx"], color='red')
plt.plot( table["t"]/nanosecond, table["my"], color='blue')
plt.plot( table["t"]/nanosecond, table["mz"], color='green')

plt.xlabel("Time (ns)")
plt.ylabel("Magnetization")

plt.show()

NameError: name 'table' is not defined

In [ ]:
def show_abs_my(m):
    my_abs = m[2,0,:,:]
    plt.figure()
    plt.imshow(my_abs, vmin=-1, vmax=1, cmap="coolwarm")
    plt.show()

show_abs_my(fields["m000199"])

In [ ]:
'''
Return the value of DW position in a cell.
(By finding a point where mz == 0)
'''
def bisect(f, x_lo, x_hi, n):
    # midpoint
    x_o = (x_lo + x_hi) / 2.0
    # base case, run enough to converge.
    if n == 0:
        return x_o
    # base case, at x_o is 0
    if f(x_o) == 0:
        return x_o
    
    # ascending (-1 --> 1)
    if f(x_lo) < f(x_hi):
        if f(x_o) > 0:
            x_o = bisect(f, x_lo, x_o, n-1)
        else:
            x_o = bisect(f, x_o, x_hi, n-1)
    # descending (1 --> -1)
    else:
        if f(x_o) < 0:
            x_o = bisect(f, x_lo, x_o, n-1)
        else:
            x_o = bisect(f, x_o, x_hi, n-1)
    return x_o


In [ ]:
def findDW(m):
    # *Global* values/ variables
    x_total = m.shape[3]
    y_total = m.shape[2]
    x_mid = int(m.shape[3]/2)

    DW_array = []
    DW_pos = []

    nDW = 0.0
    flag = False

    for iy in range(y_total):
        mx = m[0,0,iy,x_mid]
        my = m[1,0,iy,x_mid]
        mz = m[2,0,iy,x_mid]
        empty = (mx==0) and (my==0) and (mz==0)

        if empty:
            # Just got off the racetrack.
            if flag == True:
                DW_array.append(sum(DW_pos) / nDW)
                DW_pos = []
                nDW = 0
                flag = False
        else:
            # Just entered the racetrack.
            if flag == False:
                flag = True
            f = Rbf(np.arange(x_total), m[2,0,iy,:]) 
            DW_pos.append(bisect(f, 0, x_total, 15))
            nDW += 1
    
    return DW_array

In [ ]:
def findDW_A(m):
    # *Global* values/ variables
    x_total = m.shape[3]
    y_total = m.shape[2]
    x_mid = int(m.shape[3]/2)

    DW_array = []
    DW_pos = []

    nDW = 0.0
    flag = False
    count = -1

    for iy in range(y_total):
        mx = m[0,0,iy,x_mid]
        my = m[1,0,iy,x_mid]
        mz = m[2,0,iy,x_mid]
        empty = (mx==0) and (my==0) and (mz==0)

        if empty:
            # Just got off the racetrack.
            if flag == True:
                if count%2 == 0:
                    DW_array.append(sum(DW_pos) / nDW)
                DW_pos = []
                nDW = 0
                flag = False
        else:
            # Just entered the racetrack.
            if flag == False:
                flag = True
                count += 1
            f = Rbf(np.arange(x_total), m[2,0,iy,:]) 
            DW_pos.append(bisect(f, 0, x_total, 15))
            nDW += 1
    
    return DW_array

In [ ]:
def findDW_B(m):
    # *Global* values/ variables
    x_total = m.shape[3]
    y_total = m.shape[2]
    x_mid = int(m.shape[3]/2)

    DW_array = []
    DW_pos = []

    nDW = 0.0
    flag = False
    count = -1

    for iy in range(y_total):
        mx = m[0,0,iy,x_mid]
        my = m[1,0,iy,x_mid]
        mz = m[2,0,iy,x_mid]
        empty = (mx==0) and (my==0) and (mz==0)

        if empty:
            # Just got off the racetrack.
            if flag == True:
                if count%2 != 0:
                    DW_array.append(sum(DW_pos) / nDW)
                DW_pos = []
                nDW = 0
                flag = False
        else:
            # Just entered the racetrack.
            if flag == False:
                flag = True
                count += 1
            f = Rbf(np.arange(x_total), m[2,0,iy,:]) 
            DW_pos.append(bisect(f, 0, x_total, 15))
            nDW += 1
    
    return DW_array

In [ ]:
# Stack all snapshots of the magnetization on top of each other
# snaps = len(fields.keys())
# posA = np.stack([findDW_A(fields[key]) for key in iter.islice(sorted(fields.keys()), snaps-1)])
# posB = np.stack([findDW_B(fields[key]) for key in iter.islice(sorted(fields.keys()), snaps-1)])

pos = []
posA = []
posB = []

for key in sorted(fields.keys()):
    if key != "regions000000":
        pos.append(findDW(fields[key]))
        posA.append(findDW_A(fields[key]))
        print("Processing: " + str(key))
        posB.append(findDW_B(fields[key]))


In [ ]:
# Apply the two dimensional FFT
posA_fft = np.fft.fft2(posA)
posB_fft = np.fft.fft2(posB)
posA_fft = np.fft.fftshift(posA_fft)
posB_fft = np.fft.fftshift(posB_fft)

A = np.add(np.abs(posA_fft)**2, np.abs(posB_fft)**2)
# A = np.abs(posA_fft)**2
# B = np.abs(posB_fft)**2

In [ ]:
from matplotlib.colors import LogNorm

plt.figure(figsize=(10,6))

# Show the intensity plot of the 2D FFT
# extent = [ -(2*np.pi)/(2*dx), (2*np.pi)/(2*dx), -1/(2*dt), 1/(2*dt)] # extent of k values and frequencies

# plt.imshow(np.abs(posA_fft)**2, extent=extent, aspect='auto', origin='lower', cmap="inferno")
# plt.imshow(np.abs(posB_fft)**2, extent=extent, aspect='auto', origin='lower', cmap="inferno")
plt.imshow(A, aspect='auto', origin='lower', cmap="inferno", norm=LogNorm())

# plt.xlim([-2e8,2e8])
# plt.ylim([0,fmax])
plt.ylabel("$f$ (Hz)")
plt.xlabel("$k$ (1/m)")

plt.show()

In [ ]:
# Plot positions along racetracks
by_track = np.transpose(pos)

In [ ]:
print(by_track.shape)

In [ ]:
n_racetrack = range(by_track.shape[0])      # 16
t = range(by_track.shape[1])                # 201

plt.figure(figsize=(10, 6), dpi=300)

for nth in n_racetrack:
    plt.plot(t, by_track[nth], label=str(nth+1)+'th')

plt.legend(loc="upper left", fontsize='5')

In [ ]:
n_racetrack = range(by_track.shape[0])      # 16
T_vals = np.linspace(0, T, int(T/dt)+1)

plt.figure(figsize=(10, 6), dpi=300)

for nth in n_racetrack:
    plt.plot(T_vals, by_track[nth], label=str(nth+1)+'th')

plt.legend(loc="upper left", fontsize='5')